## Data preprocess

In [67]:
from pathlib import Path
from openpyxl import Workbook # type: ignore
from pathlib import Path
import os


def to_excel(data, filename, sheet_style):
    workbook = Workbook()
    sheet = workbook.active

    # | user-response-time | request-number | response-ip   | process-time  |
    # | 0.3                | 10000          | 192.168.0.150 | 14.523432     |
    # | 0.5                | 20000          | 192.168.0.151 | 9.5232642     |

    sheet.append(sheet_style)

    for row in data:
        sheet.append(row)

    result_dir_path = Path.cwd().parent / "results" / "result_v3"

    if not os.path.exists(result_dir_path):
        os.makedirs(result_dir_path, exist_ok=True)

    workbook.save(filename=f"{result_dir_path}\\{filename}.xlsx")


# read input dataset

filename_prefix = "clientv5#loops1#requests_batch150#Thu-Jul-25-22-55-54-2024"
dataset_read_filename = filename_prefix
training_data_dir = Path.cwd().parent / "training_data" / "data_set3"
# Data preprocessing
file_path = f"{training_data_dir}\\{dataset_read_filename}.xlsx"


# set result output filename and path
result_suffix = "result"
result_output_filename = f"{filename_prefix}{result_suffix}"

In [68]:

import pandas as pd # type: ignore
import numpy as np # type: ignore


def read_data(filename):
    df = pd.read_excel(filename)

    columns = df.columns.to_list()

    data_dict = {col: df[col].to_list() for col in columns}

    return data_dict


def data_preprocess(filepath):
    data = read_data(filepath)
    # TODO more...

    # to numpy
    for key in data.keys():
        data[key] = np.array(data[key])

    return data

In [69]:
dataset = data_preprocess(file_path)

## DATA Style View

In [70]:
# # dataset is a dictionary

for col_header, col_data_list in dataset.items():
    print(col_header, col_data_list)

num [500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 500000 500000 500000 500000 500000 500000 500000 500000 500000 500000
 5

## XGBOOST - 1

In [71]:
from xgboost import XGBRegressor # type: ignore

# Create XGBoost regression model
model = XGBRegressor(
    objective="reg:squarederror",   # Loss function to minimize: squared error for regression  损失函数为平方损失函数, 
    n_estimators=100,  # Number of boosting rounds (number of trees)  迭代次数，即基础学习器的数量
    learning_rate=0.1,  # Boosting learning rate (controls the step size)  学习率，控制每次更新的步长
    max_depth=5,  # Maximum depth of a tree  树的最大深度
    min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child  叶子节点最小权重
    subsample=0.8,  # Subsample ratio of the training instance (randomly sampled)  每棵树随机采样的比例
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree  每棵树随机选择的特征比例
    alpha=0.01,  # L1 regularization term on weights  L1 正则化项参数
    reg_lambda=0.01,  # L2 regularization term on weights  L2 正则化项参数
)

In [72]:
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.metrics import mean_squared_error # type: ignore

# # select value from dataset
X = np.array([dataset.get("num"), dataset.get("total_response_time"), dataset.get("processing_tasks_in_worker_node")]).T
y = dataset.get("process_in_manager_node")

# train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=35
)

# model fit
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [73]:
from sklearn.metrics import mean_absolute_error # type: ignore

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 19.09030719609031
Mean Absolute Error: 3.668565599123624
    real    |    pred   

    207.5659682750702     |     205.5228729248047      
    312.9886300563812     |     309.552490234375      
    419.3079333305359     |     421.38812255859375      
    188.7104740142822     |     181.09666442871094      
    337.7047772407532     |     331.9804992675781      
    221.633208990097     |     224.1815948486328      
    395.4064800739288     |     387.74755859375      
    181.0993525981903     |     180.30003356933594      
    190.9630877971649     |     188.34400939941406      
    164.6196267604828     |     160.8728790283203      
    411.8957686424255     |     410.3179626464844      
    389.16481590271     |     387.74755859375      
    48.68064856529236     |     52.107059478759766      
    378.1270010471344     |     373.4290771484375      
    124.8623402118683     |     127.30049133300781      
    413.9221587181091     |     411.1271057128906      
   

## XGBRegressor

In [74]:
from xgboost import XGBRegressor # type: ignore
from sklearn.metrics import mean_squared_error # type: ignore
from sklearn.model_selection import GridSearchCV # type: ignore


# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    tree_method="hist",
    device="cuda",
)

# Define the parameter grid for GridSearchCV
param_grid = {
    "max_depth": [3],  # Range of maximum depth of trees to explore  扩展最大深度的范围
    "learning_rate": [0.1],  # Range of learning rates to explore  扩展学习率的范围
    "n_estimators": [100],  # Range of number of trees (boosting rounds) to explore  扩展树的数量范围
    "gamma": [0],   # Range of gamma parameter to explore (controls tree split) 扩展 gamma 参数范围
    "subsample": [1],  # Range of subsample ratios to explore  扩展子样本比例范围
    "colsample_bytree": [0.8, 0.7],  # Range of column subsample ratios for each tree to explore  扩展列采样比例范围
    "reg_alpha": [0, 0.5],  # Range of L1 regularization parameters to explore  扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.5],  # Range of L2 regularization parameters to explore  扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found:\n")
for k, v in grid_search.best_params_.items():
    print('\t', k, ":", v)


# 使用最佳参数重新训练模型
# Re-train the model using the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, verbose=True)

Best parameters found:

	 colsample_bytree : 0.8
	 gamma : 0
	 learning_rate : 0.1
	 max_depth : 3
	 n_estimators : 100
	 reg_alpha : 0
	 reg_lambda : 0
	 subsample : 1


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [75]:
preds = best_model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")


avg = np.mean(y_test)

print(f"AVG: {avg}")

# write into excal

sheet_style = ["test", "prediction", "difference", "accuracy", "is_waited"]

data_list = list()

is_waited = True


for i in range(len(y_test)):
    is_waited = True
    acc = 0
    diff = round(abs(y_test[i] - preds[i]), 6)
    if preds[i] < 0:
        acc = 0
    elif diff < 1:
        acc = 1
        is_waited = False
    else:
        rate = round(diff / y_test[i], 4)
        if rate <= 1 or rate >= 0:
            acc = 1 - rate
        if rate > 1:
            acc = 0

    data_list.append(
        [
            y_test[i],
            preds[i],
            diff,
            f"{round(acc * 100, 2)}%",
            is_waited
        ],
    )


to_excel(data_list, result_output_filename, sheet_style)

print("Results saved finished.")

RMSE: 3.9121903200007315
MAE: 3.2198048035303644
AVG: 240.03563713232677
Results saved finished.


d:\model_fit\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
